<a href="https://colab.research.google.com/github/bryanbayup/petpoint/blob/main/PetpointWithGeneralConver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi

In [2]:
import re
import string
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

Membaca Dataset dan Preprocessing Teks

In [3]:
# Membaca dataset penyakit
df_disease = pd.read_csv('disease_classification.csv')

In [4]:
# Preprocessing dataset
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [5]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_text = stopword_remover.remove(text)
    return cleaned_text

In [6]:
df_disease['Cleaned_Gejala'] = df_disease['Gejala'].apply(clean_text).apply(remove_stopwords)
df_disease['Cleaned_Penanganan'] = df_disease['Penanganan Pertama'].apply(clean_text).apply(remove_stopwords)

In [7]:
# Mengubah nama penyakit menjadi label numerik
le_disease = LabelEncoder()
df_disease['Label_Penyakit'] = le_disease.fit_transform(df_disease['Nama Penyakit'])

In [8]:
# Menggabungkan teks gejala untuk penyakit
all_text_disease = df_disease['Cleaned_Gejala'].tolist()
vectorizer_disease = TfidfVectorizer()
X_disease = vectorizer_disease.fit_transform(all_text_disease)
y_disease = df_disease['Label_Penyakit']

In [9]:
# Membagi data untuk penyakit
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(
    X_disease, y_disease, test_size=0.2, random_state=42)

In [10]:
# Membangun model untuk penyakit
model_disease = Sequential()
model_disease.add(Dense(128, input_shape=(X_train_d.shape[1],), activation='relu'))
model_disease.add(Dropout(0.5))
model_disease.add(Dense(64, activation='relu'))
model_disease.add(Dense(len(le_disease.classes_), activation='softmax'))
model_disease.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
model_disease.fit(X_train_d, y_train_d,
                  epochs=50,
                  batch_size=64,
                  validation_data=(X_test_d, y_test_d))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 371ms/step - accuracy: 0.2004 - loss: 2.2364 - val_accuracy: 0.5250 - val_loss: 2.1485
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5078 - loss: 2.1207 - val_accuracy: 0.7500 - val_loss: 2.0345
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.7086 - loss: 2.0169 - val_accuracy: 1.0000 - val_loss: 1.9214
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.8195 - loss: 1.9108 - val_accuracy: 1.0000 - val_loss: 1.8028
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.8230 - loss: 1.8062 - val_accuracy: 1.0000 - val_loss: 1.6741
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.8840 - loss: 1.6803 - val_accuracy: 1.0000 - val_loss: 1.5349
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.8941 - loss: 1.5699 - val_accuracy: 1.0000 - val_loss: 1.3887
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.9445 - loss: 1.4154 - val_accuracy: 1.0000 - val_loss: 1

In [11]:
# Dataset percakapan umum
df_conversation = pd.read_csv('general_conversation.csv')

In [12]:
# Preprocessing dataset percakapan umum
df_conversation['Cleaned_Input'] = df_conversation['Input'].apply(clean_text).apply(remove_stopwords)

In [13]:
# Membuat mapping dictionary untuk percakapan umum
conversation_dict = dict(zip(df_conversation['Cleaned_Input'], df_conversation['Output']))

In [14]:
def get_general_response(user_input):
    processed_input = clean_text(user_input)
    processed_input = remove_stopwords(processed_input)
    response = conversation_dict.get(processed_input, None)
    return response

In [15]:
# Fungsi deteksi hewan
def detect_animal_type(user_input):
    animal_types = ['kucing', 'anjing']
    for animal in animal_types:
        if animal in user_input.lower():
            return animal
    return None

In [16]:
# Daftar gejala
symptoms_keywords = ['muntah', 'demam', 'batuk', 'diare', 'kejang', 'lemas',
                     'pilek', 'bersin', 'luka', 'lesu', 'gatal', 'tidak nafsu makan', 'dehidrasi']

In [17]:
def is_symptom_input(user_input):
    return any(keyword in user_input.lower() for keyword in symptoms_keywords)

In [18]:
def is_no_more_symptoms(user_input):
    negations = ['tidak', 'nggak', 'enggak', 'gak', 'ga', 'udah', 'cukup', 'sudah', 'tidak ada']
    return any(word in user_input.lower() for word in negations)

In [19]:
# Variabel global status percakapan
current_animal = None
symptoms_list = []
conversation_state = 'initial'

In [20]:
def chatbot_response(user_input):
    global current_animal, symptoms_list, conversation_state

    # Preprocessing input pengguna
    processed_input = clean_text(user_input)
    processed_input = remove_stopwords(processed_input)

    if conversation_state == 'asking_symptoms':
        if is_no_more_symptoms(user_input):
            if symptoms_list:
                combined_symptoms = ' '.join(symptoms_list)
                vectorized_symptoms = vectorizer_disease.transform([combined_symptoms])
                prediction = model_disease.predict(vectorized_symptoms)
                predicted_label = prediction.argmax(axis=-1)[0]
                disease_name = le_disease.inverse_transform([predicted_label])[0]
                df_animal = df_disease[df_disease['Nama Hewan'].str.lower() == current_animal]
                first_aid = df_animal[df_animal['Nama Penyakit'] == disease_name]['Penanganan Pertama'].iloc[0]
                response = f"Berdasarkan gejala yang Anda sebutkan, sepertinya {current_animal} Anda mengalami {disease_name}. {first_aid}"
                # Reset percakapan
                conversation_state = 'initial'
                current_animal = None
                symptoms_list = []
                return response
            else:
                conversation_state = 'initial'
                current_animal = None
                symptoms_list = []
                return "Anda tidak menyebutkan gejala apapun. Ada yang bisa saya bantu?"
        else:
            symptoms_list.append(user_input)
            return "Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'."
    elif conversation_state == 'asking_animal':
        animal = detect_animal_type(user_input)
        if animal:
            current_animal = animal
            conversation_state = 'asking_symptoms'
            return f"Apa gejala yang dialami {current_animal} Anda?"
        else:
            return "Maaf, saya hanya bisa memberikan saran untuk kucing atau anjing. Hewan apa yang Anda miliki?"
    else:
        # Cek apakah input adalah percakapan umum
        response = get_general_response(user_input)
        if response:
            return response
        # Cek apakah input terkait dengan hewan sakit
        elif 'sakit' in user_input.lower() or is_symptom_input(user_input):
            animal = detect_animal_type(user_input)
            if animal:
                current_animal = animal
                conversation_state = 'asking_symptoms'
                return f"Apa gejala yang dialami {current_animal} Anda?"
            else:
                conversation_state = 'asking_animal'
                return "Hewan apa yang Anda miliki? (kucing/anjing)"
        else:
            return "Maaf, saya tidak mengerti. Bisa dijelaskan lebih lanjut?"

In [ ]:
# Loop interaksi dengan pengguna
print("Chatbot: Halo, ada yang bisa saya bantu?")
while True:
    user_input = input("Anda: ")
    if user_input.lower() in ['exit', 'quit', 'keluar', 'bye', 'sampai jumpa']:
        print("Chatbot: Terima kasih, semoga membantu!")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)

Chatbot: Halo, ada yang bisa saya bantu?
Anda: halo
Chatbot: Halo! Ada yang bisa saya bantu?
Anda: siapa kamu
Chatbot: Saya adalah Alexa, chatbot yang siap membantu Anda.
Anda: saya ingin bertanya
Chatbot: Maaf, saya tidak mengerti. Bisa dijelaskan lebih lanjut?
Anda: kucing saya sakit
Chatbot: Apa gejala yang dialami kucing Anda?
Anda: mual
Chatbot: Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'.
Anda: demam
Chatbot: Apakah ada gejala lain yang dialami hewan Anda? Jika tidak ada, silakan katakan 'tidak ada'.
Anda: tidak ada
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Chatbot: Berdasarkan gejala yang Anda sebutkan, sepertinya kucing Anda mengalami Feline Leukemia Virus. Isolasi kucing yang terinfeksi, berikan perawatan suportif, konsultasi dokter hewan. Pastikan untuk konsultasi dengan dokter hewan.
